# 🎌 Anime 4K Upscale - ONE CLICK

1. Sửa đường dẫn ở Cell 1
2. Nhấn **Runtime → Run all** (`Ctrl+F9`)

In [ ]:
#@title ⚙️ CẤU HÌNH
INPUT_DIR = "/content/drive/MyDrive/Movie/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movie/Output" #@param {type:"string"}
MODE = "anime_4k" #@param ["anime_4k", "anime_fast", "ultra_fast"]
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}

In [ ]:
#@title 🚀 CHẠY
import os, glob, subprocess, json, time, pickle
from datetime import timedelta

# Reset về /content trước
os.chdir('/content')

# Mount
print("📂 Mounting Drive...")
from google.colab import drive
drive.mount('/content/drive')

# GPU
print("\n🖥️ GPU:")
!nvidia-smi --query-gpu=name,memory.free --format=csv

# ====== INSTALL ======
print("\n📦 Installing...")
os.chdir('/content')

ESRGAN_PATH = '/content/Real-ESRGAN'
MODEL_PATH = f'{ESRGAN_PATH}/weights/realesr-animevideov3.pth'

if not os.path.exists(MODEL_PATH):
    # Xóa folder cũ nếu có
    if os.path.exists(ESRGAN_PATH):
        !rm -rf {ESRGAN_PATH}
    
    # Clone mới
    print("   Cloning Real-ESRGAN...")
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    
    # Vào folder
    os.chdir(ESRGAN_PATH)
    
    # Install dependencies với phiên bản tương thích
    print("   Installing dependencies...")
    !pip install torch==2.0.1 torchvision==0.15.2 --quiet
    !pip install basicsr==1.4.2 facexlib gfpgan --quiet
    !pip install -r requirements.txt --quiet
    !python setup.py develop 2>&1 | tail -1
    
    # Download model
    print("   Downloading model...")
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q

print("✅ Tools ready!")

# Config
MODES = {
    'anime_4k': {'tile': 192, 'tile_pad': 24, 'crf': 22, 'preset': 'fast'},
    'anime_fast': {'tile': 128, 'tile_pad': 16, 'crf': 24, 'preset': 'veryfast'},
    'ultra_fast': {'tile': 64, 'tile_pad': 8, 'crf': 28, 'preset': 'ultrafast'}
}
cfg = MODES[MODE]
PROGRESS = f"/content/drive/MyDrive/Movie/.progress_{WORKER_ID}.pkl"

# ======= DEBUG =======
print(f"\n🔍 Kiểm tra thư mục:")
print(f"📂 INPUT: {INPUT_DIR} - Exists: {os.path.exists(INPUT_DIR)}")

if os.path.exists(INPUT_DIR):
    for item in os.listdir(INPUT_DIR):
        full = os.path.join(INPUT_DIR, item)
        if os.path.isdir(full):
            print(f"   📁 {item}")
        else:
            print(f"   📄 {item} ({os.path.getsize(full)/(1024**2):.0f}MB)")
else:
    print("   ❌ Không tìm thấy!")
    !find /content/drive/MyDrive -maxdepth 3 -type d -name "*Input*" 2>/dev/null

# ======= SCAN =======
print(f"\n🔍 Scanning...")

def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

done = set()
if os.path.exists(PROGRESS):
    with open(PROGRESS, 'rb') as f: done = pickle.load(f)

all_files = []
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        for f in files: all_files.append(os.path.join(root, f))

VIDEO_EXT = ('.mp4', '.mkv', '.avi', '.mov', '.webm')
all_vids = sorted([f for f in all_files if f.lower().endswith(VIDEO_EXT)])
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    
    if name in done or (os.path.exists(out) and os.path.getsize(out) > 10*1024*1024):
        print(f"⏭️ {name}")
    elif info['w'] >= 3800:
        print(f"⏭️ [4K] {name}")
    elif info['w'] > 0:
        to_process.append({'path': p, 'name': name, 'info': info})
        print(f"✅ {name} ({info['w']}x{info['h']})")

print(f"\n📊 {len(to_process)} videos cần xử lý")

# ======= PROCESS =======
if not to_process:
    print("✅ Không có video nào cần xử lý!")
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs('/content/temp', exist_ok=True)
    
    print(f"\n🚀 BẮT ĐẦU - Mode: {MODE}")
    
    for idx, item in enumerate(to_process, 1):
        p, name, info = item['path'], item['name'], item['info']
        out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
        temp = f'/content/temp/{idx}.mp4'
        
        print(f"\n🎌 [{idx}/{len(to_process)}] {name}")
        start = time.time()
        
        try:
            # Upscale
            os.chdir(ESRGAN_PATH)
            cmd = f'python inference_realesrgan_video.py -n realesr-animevideov3 -i "{p}" -o "{temp}" -s 2 --tile {cfg["tile"]} --tile_pad {cfg["tile_pad"]} --suffix "" --fp32'
            os.system(cmd)
            
            if not os.path.exists(temp):
                raise Exception("Upscale failed")
            
            # Encode
            enc_cmd = f'ffmpeg -y -i "{p}" -i "{temp}" -map 1:v -map 0:a? -map 0:s? -c:v libx265 -crf {cfg["crf"]} -preset {cfg["preset"]} -tag:v hvc1 -c:a copy -c:s copy -map_metadata 0 -map_chapters 0 "{out}" -loglevel warning -stats'
            os.system(enc_cmd)
            
            # Save progress
            done.add(name)
            with open(PROGRESS, 'wb') as f: pickle.dump(done, f)
            
            elapsed = time.time() - start
            size = os.path.getsize(out) / (1024**3)
            print(f"✅ Done: {timedelta(seconds=int(elapsed))} | {size:.2f}GB")
            
        except Exception as e:
            print(f"❌ Error: {e}")
        finally:
            if os.path.exists(temp): os.remove(temp)
    
    print(f"\n🏁 XONG! Output: {OUTPUT_DIR}")